In [1]:
from numpy import array
from keras.callbacks import CSVLogger
import numpy as np
from pickle import dump
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import GRU
from keras.layers import Embedding
import io

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
def load_doc(filename):
    with io.open(filename, 'r', encoding='utf8') as f:
        text = f.read()
    return text

In [4]:
file_path = "/content/drive/My Drive/next-word-prediction/data/data5.txt"
doc = load_doc(file_path)
lines = doc.split('\n')

In [5]:
lines

['a guide to word embedding with',
 'guide to word embedding with gensim',
 'to word embedding with gensim model',
 'word embedding with gensim model handson',
 'embedding with gensim model handson graph',
 'with gensim model handson graph neural',
 'gensim model handson graph neural networks',
 'model handson graph neural networks with',
 'handson graph neural networks with pytorch',
 'graph neural networks with pytorch pytorch',
 'neural networks with pytorch pytorch geometric',
 'networks with pytorch pytorch geometric how',
 'with pytorch pytorch geometric how to',
 'pytorch pytorch geometric how to use',
 'pytorch geometric how to use in',
 'geometric how to use in python',
 'how to use in python databricks',
 'to use in python databricks how',
 'use in python databricks how to',
 'in python databricks how to save',
 'python databricks how to save files',
 'databricks how to save files in',
 'how to save files in csv',
 'to save files in csv on',
 'save files in csv on your',
 'fi

In [6]:
# integer encode sequences of words
tokenizer = Tokenizer()
tokenizer.fit_on_texts(lines)
sequences = tokenizer.texts_to_sequences(lines)
# vocabulary size
vocab_size = len(tokenizer.word_index) + 1

In [7]:
vocab_size

7761

In [8]:
sequences

[[3, 58, 2, 433, 1730, 11],
 [58, 2, 433, 1730, 11, 2310],
 [2, 433, 1730, 11, 2310, 86],
 [433, 1730, 11, 2310, 86, 3472],
 [1730, 11, 2310, 86, 3472, 674],
 [11, 2310, 86, 3472, 674, 69],
 [2310, 86, 3472, 674, 69, 90],
 [86, 3472, 674, 69, 90, 11],
 [3472, 674, 69, 90, 11, 301],
 [674, 69, 90, 11, 301, 301],
 [69, 90, 11, 301, 301, 1731],
 [90, 11, 301, 301, 1731, 5],
 [11, 301, 301, 1731, 5, 2],
 [301, 301, 1731, 5, 2, 56],
 [301, 1731, 5, 2, 56, 7],
 [1731, 5, 2, 56, 7, 35],
 [5, 2, 56, 7, 35, 3473],
 [2, 56, 7, 35, 3473, 5],
 [56, 7, 35, 3473, 5, 2],
 [7, 35, 3473, 5, 2, 232],
 [35, 3473, 5, 2, 232, 1409],
 [3473, 5, 2, 232, 1409, 7],
 [5, 2, 232, 1409, 7, 2311],
 [2, 232, 1409, 7, 2311, 17],
 [232, 1409, 7, 2311, 17, 8],
 [1409, 7, 2311, 17, 8, 855],
 [7, 2311, 17, 8, 855, 233],
 [2311, 17, 8, 855, 233, 3],
 [17, 8, 855, 233, 3, 856],
 [8, 855, 233, 3, 856, 434],
 [855, 233, 3, 856, 434, 4],
 [233, 3, 856, 434, 4, 1732],
 [3, 856, 434, 4, 1732, 1173],
 [856, 434, 4, 1732, 1173, 

In [9]:
length = 6

In [10]:
sequences = array(sequences)
sequences

array([[   3,   58,    2,  433, 1730,   11],
       [  58,    2,  433, 1730,   11, 2310],
       [   2,  433, 1730,   11, 2310,   86],
       ...,
       [ 643,    5,    2,   55,    3,   74],
       [   5,    2,   55,    3,   74,   47],
       [   2,   55,    3,   74,   47,  673]])

In [11]:
c=1
h=0
while(c!=0):
    limit = len(sequences)
    d =[]
    c=0
    for i in range(0 , limit):
        l=len(sequences[i])
        if(l!=length):
            print(sequences[i])
            sequences = np.delete(sequences, i, 0)
            
            c=1
            print("error at ", i)
            print("Deleting :_________________________ ") 
            h = h+1
            break        
print("Total deletion: ", h)

Total deletion:  0


In [12]:
# separate into input and output
sequences = array(sequences)
X, y = sequences[:,:-1], sequences[:,-1]
y = to_categorical(y, num_classes=vocab_size)
seq_length = X.shape[1]

In [13]:
# define model
model = Sequential()
model.add(Embedding(vocab_size, 50, input_length=seq_length))
model.add(GRU(100, return_sequences=True))
model.add(GRU(100))
model.add(Dense(100, activation='relu'))
model.add(Dense(vocab_size, activation='softmax'))
print(model.summary())
# compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit model
csv_logger = CSVLogger('training_1_1.log')
model.fit(X, y, batch_size=500, epochs=1000, callbacks=[csv_logger])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 5, 50)             388050    
                                                                 
 gru (GRU)                   (None, 5, 100)            45600     
                                                                 
 gru_1 (GRU)                 (None, 100)               60600     
                                                                 
 dense (Dense)               (None, 100)               10100     
                                                                 
 dense_1 (Dense)             (None, 7761)              783861    
                                                                 
Total params: 1,288,211
Trainable params: 1,288,211
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/1000
98/98 [=====================

In [14]:
model.save('/content/drive/My Drive/next-word-prediction/models/model_5.h5')
dump(tokenizer, open(r'/content/drive/My Drive/next-word-prediction/models/tokenizer_5.pkl', 'wb'))